In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

In [ ]:
data =pd.read_csv('https://raw.githubusercontent.com/adil200/Medical-Diagnoser/main/medical_data.csv')
data.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


In [ ]:
tokenizer=Tokenizer(num_words=5000 ,oov_token="<OOV>")
tokenizer.fit_on_texts(data['Patient_Problem'])
sequences= tokenizer.texts_to_sequences(data['Patient_Problem'])

In [ ]:
max_length=max(len(x) for x in sequences)
padded_sequences =pad_sequences(sequences,maxlen=max_length,padding='post')

In [ ]:
Label_Encoder_Disease=LabelEncoder()
Label_Encoder_Prescription=LabelEncoder()
Disease_lable=Label_Encoder_Disease.fit_transform(data['Disease'])
prescription_lable=Label_Encoder_Prescription.fit_transform(data['Prescription'])
disease_categorical=to_categorical(Disease_lable)
prescription_categorical=to_categorical(prescription_lable)

In [ ]:
Y=np.hstack((disease_categorical,prescription_categorical))

In [ ]:
Input_layer=Input(shape=(max_length,))
embedding=Embedding(input_dim=5000,output_dim=64)(Input_layer)
first_layer=LSTM(64)(embedding)
disease_output=Dense(len(Label_Encoder_Disease.classes_),activation='softmax',name='disease_output')(first_layer)
prescription_output=Dense(len(Label_Encoder_Prescription.classes_),activation='softmax',name='prescription_output')(first_layer)

In [ ]:
model=Model(inputs=Input_layer,outputs=[disease_output,prescription_output])
model.compile(loss={'disease_output':'categorical_crossentropy','prescription_output':'categorical_crossentropy'},
optimizer='adam',
metrics={'disease_output':['accuracy'],'prescription_output':['accuracy']}
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 17)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 17, 64)         │        320,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 64)             │         33,024 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ disease_output (Dense)    │ (None, 178)            │         11,570 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ prescription_output       │ (None, 388)            │         25,220 │ lstm[0][0]             │
│ (Dense)                   │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 389,814 (1.49 MB)

 Trainable params: 389,814 (1.49 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(padded_sequences,[disease_categorical,prescription_categorical],epochs=100,batch_size=32)

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - disease_output_accuracy: 0.0124 - loss: 11.1427 - prescription_output_accuracy: 0.0000e+00
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - disease_output_accuracy: 0.0476 - loss: 11.1198 - prescription_output_accuracy: 0.0050
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - disease_output_accuracy: 0.0400 - loss: 11.0666 - prescription_output_accuracy: 0.0000e+00
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - disease_output_accuracy: 0.0369 - loss: 10.9287 - prescription_output_accuracy: 0.0099
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - disease_output_accuracy: 0.0321 - loss: 10.8469 - prescription_output_accuracy: 0.0165
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - disease_output_accuracy: 0.0405 - loss: 10.7045 - prescription_output_accuracy: 0.0185
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - disease_output_accuracy: 0.0395 - loss: 10.5316 - prescription_output_accuracy: 0.0156
Epoch 

In [ ]:
def make_prediction(input_problem):
  sequences=tokenizer.texts_to_sequences([input_problem])
  padded_sequences=pad_sequences(sequences,maxlen=max_length,padding='post')
  predictions=model.predict(padded_sequences)
  disease_prediction=Label_Encoder_Disease.inverse_transform([np.argmax(predictions[0])])
  prescription_prediction=Label_Encoder_Prescription.inverse_transform([np.argmax(predictions[1])])
  return disease_prediction[0],prescription_prediction[0]
input="I've experienced a loss of appetite and don't enjoy food anymore ."
make_prediction(input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step


('Depression', 'Antidepressants; eating nutrient-rich foods.')